In [60]:
import requests
from bs4 import BeautifulSoup

In [61]:
def obtener_contenido_pagina(url):
    respone=requests.get(url)
    return respone.text

In [62]:
def analizar_contenido_html(html):
    return BeautifulSoup(html,'html.parser')

In [63]:
data = []

def procesar_pagina(soup):

    titulos = []
    precios_originales = []
    precios_descuento = []
    precios_envio = []
    skus = []
    stocks = []

    titulo_items = soup.find_all('div', class_="emproduct_right")

    for item in titulo_items:
        titulo = item.a.text.strip()
        titulos.append(titulo)

    precio_original_items = soup.find_all("span", class_="oldPrice")

    for item in precio_original_items:
        precio_original = item.text.strip()
        precios_originales.append(precio_original)

    precio_descuento_items = soup.find_all("div", class_="emproduct_right_price_left")

    for item in precio_descuento_items:
        precio_descuento = item.label.text.strip()
        precios_descuento.append(precio_descuento)

    sku_items = soup.find_all("div", class_="emproduct_right_artnum")

    for item in sku_items:
        sku = item.text.strip()
        skus.append(sku.lstrip("SKU: "))

    precio_envio_items = soup.find_all("span", class_={"deliveryvalue", "deliverytextfree"})

    for item in precio_envio_items:
        precio_envio = item.text
        precios_envio.append(precio_envio)

    stock_items = soup.find_all("div", class_="emstock")

    for item in stock_items:
        stock = item.text.split()
        stocks.append(stock[1])

    min_length = min(len(titulos), len(skus), len(precios_originales), len(precios_descuento), len(precios_envio), len(stocks))
    
    for i in range(min_length):
        data.append({
            "Título": titulos[i] if i < len(titulos) else None,
            "SKU": skus[i]  if i < len(skus) else None,
            "Precio Original": precios_originales[i]  if i < len(precios_originales) else None,
            "Precio Descuento": precios_descuento[i] if i < len(precios_descuento) else None,
            "Precio Envío": precios_envio[i] if i < len(precios_envio) else None,
            "Stock": stocks[i] if i < len(stocks) else None
        })


In [64]:
def manejar_paginacion(url_base,num_paginas):
    for i in range (1,num_paginas+1):
        url= url_base+str(i)+"/"
        contenido_pagina=obtener_contenido_pagina(url)
        soup=analizar_contenido_html(contenido_pagina)
        procesar_pagina(soup)


In [65]:
url_base="https://www.cyberpuerta.mx/Promociones/"
num_paginas=10

manejar_paginacion(url_base,num_paginas)

In [66]:
import pandas as pd

df=pd.DataFrame(data)


In [68]:
df

,Título,SKU,Precio Original,Precio Descuento,Precio Envío,Stock
0,"Procesador AMD Ryzen 7 5800X3D, S-AM4, 3.40GHz...",100-100000651WOF,"$7,013.00","$5,909.00",$99.00,7
1,"Procesador Intel Core i5-12400F Sin Gráficos, ...",BX8071512400F,"$2,933.00","$2,649.00",$99.00,11
2,"Ubiquiti Networks Access Point Lite AP AC, 450...",LAP-GPS,"$2,041.00","$1,769.00",$99.00,375
3,"Memoria RAM HPE DDR4, 2933MHz, 32GB, CL21, Reg...",P00924-B21,"$11,101.00","$2,029.00",$99.00,25
4,"Memoria RAM HPE DDR4, 2933MHz, 16GB, CL21, Dua...",P00922-B21,"$20,959.00",$659.00,$99.00,10
...,...,...,...,...,...,...
155,"Monitor BenQ GW2283 LED 21.5"", Full HD, HDMI, ...",9H.LHLLA.TBL,"$2,163.00","$1,879.00",$149.00,29
156,"Monitor AOC 22B2HN LED 21.5"", Full HD, 75Hz, H...",22B2HN,"$1,970.00","$1,519.00",$144.00,49
157,"Monitor Gamer ASUS VA24DQ LED 23.8"", Full HD, ...",90LM0543-B013B0,"$3,032.00","$2,699.00",$154.00,3
158,Mouse Gamer Ergonómico Logitech Óptico G502 He...,910-005550,"$1,057.00",$909.00,$99.00,¡1


In [67]:
import datetime

fecha_actual=datetime.datetime.now().strftime("%d-%m-%Y")
df.to_csv(f"dataset/Promociones-Cyberpuerta-{fecha_actual}.csv",index=False)